In [1]:
import sqlite3
import pandas as pd

In [2]:
db_file=r'reddit_comments.db'
conn = sqlite3.connect(db_file)

In [3]:
input=pd.read_sql("SELECT * FROM rc WHERE pcomment IS NOT NULL LIMIT 20",conn)

In [4]:
for index,row in input.iterrows():
    print('parent: ',row['pcomment'])
    print('comment: ',row['comment'])
    print('#######################')

parent:  Yeah, probably. 
comment:  If she can't say yes on the spot, I'd legit reconsider the proposal. 
#######################
parent:  I legitimately have no idea what's going on, but am okay with it.  Also someone changed my text flair &amp;#3232;_&amp;#3232;
comment:  Me neither
#######################
parent:  And sorta not really even a doctor.
comment:  He was a great Dr. Just not for humans.  
#######################
parent:  So, fat dudes and feminists wanted to circlejerk in person?
comment:  That's offensive to any normal fat guy.
#######################
parent:  If that epiphany ever comes, we probably won't see a rant about it.
comment:  We'll see a lack of rants.
#######################
parent:  'sonuvabitch' -vapingwithtwisted420  That compilation video someone posted had me in tears. 
comment:  Oh I know I too was dying. Now every time he says that in a video I crack up!
#######################
parent:  Between *Dawn*, *New Horizons*, *Akatsuki*, the end of MESSENGER,

In [5]:
input=pd.read_sql("SELECT * FROM rc WHERE pcomment IS NOT NULL LIMIT 10000",conn)

In [11]:
#converting text to lowercase
def lower_txt(sentence):
    return sentence.lower()
input['pcomment']=input['pcomment'].apply(lower_txt)
input['comment']=input['comment'].apply(lower_txt)

In [12]:
input.head()

,parent_id,comment_id,pcomment,comment,score,subreddit
0,t1_cnbkf1f,t1_cnbn4j0,"yeah, probably.","if she can't say yes on the spot, i'd legit re...",11,AskMen
1,t1_cnbkla9,t1_cnbn4j6,"i legitimately have no idea what's going on, b...",me neither,3,twitchplayspokemon
2,t1_cnblwj8,t1_cnbn4jc,and sorta not really even a doctor.,he was a great dr. just not for humans.,4,WTF
3,t1_cnbhvpr,t1_cnbn4jj,"so, fat dudes and feminists wanted to circleje...",that's offensive to any normal fat guy.,9,TumblrInAction
4,t1_cnbmy2p,t1_cnbn4js,"if that epiphany ever comes, we probably won't...",we'll see a lack of rants.,9,anime


In [13]:
#tokenizing text using treebank tokenizer
from nltk.tokenize import TreebankWordTokenizer

def token(sentence):
    sentence = TreebankWordTokenizer().tokenize(sentence)
    sentence = ' '.join(sentence)
    return sentence

input['pcomment']=input['pcomment'].apply(token)
input['comment']=input['comment'].apply(token)

In [14]:
input.head()

,parent_id,comment_id,pcomment,comment,score,subreddit
0,t1_cnbkf1f,t1_cnbn4j0,"yeah , probably .","if she ca n't say yes on the spot , i 'd legit...",11,AskMen
1,t1_cnbkla9,t1_cnbn4j6,"i legitimately have no idea what 's going on ,...",me neither,3,twitchplayspokemon
2,t1_cnblwj8,t1_cnbn4jc,and sorta not really even a doctor .,he was a great dr. just not for humans .,4,WTF
3,t1_cnbhvpr,t1_cnbn4jj,"so , fat dudes and feminists wanted to circlej...",that 's offensive to any normal fat guy .,9,TumblrInAction
4,t1_cnbmy2p,t1_cnbn4js,"if that epiphany ever comes , we probably wo n...",we 'll see a lack of rants .,9,anime


In [16]:
import collections
parent_word_counter = collections.Counter([word for sentence in input['pcomment'] for word in sentence.split()])
comment_words_counter = collections.Counter([word for sentence in input['comment'] for word in sentence.split()])

print('{} parent words.'.format(len([word for sentence in input['pcomment'] for word in sentence.split()])))
print('{} unique parent words.'.format(len(parent_word_counter)))
print('10 Most common words in the parent dataset:')
print('"' + '" "'.join(list(zip(*parent_word_counter.most_common(10)))[0]) + '"')
print()
print('{} comment words.'.format(len([word for sentence in input['comment'] for word in sentence.split()])))
print('{} unique comment words.'.format(len(comment_words_counter)))
print('10 Most common words in the comment dataset:')
print('"' + '" "'.join(list(zip(*comment_words_counter.most_common(10)))[0]) + '"')

177997 parent words.
20843 unique parent words.
10 Most common words in the parent dataset:
"the" "." "," "i" "a" "to" "it" "you" "and" "?"

162195 comment words.
19442 unique comment words.
10 Most common words in the comment dataset:
"." "the" "i" "," "a" "to" "it" "and" "that" "you"


In [21]:
#converting words to integeres from 1 to vocabulary length
from keras.preprocessing.text import Tokenizer

def tokenizer(data):
    tokenizer=Tokenizer(filters='')
    tokenizer.fit_on_texts(data)
    t=tokenizer.texts_to_sequences(data)
    return t, tokenizer

In [29]:
X,ptokens=tokenizer(input['pcomment'])

In [30]:
X[0:2]

[[138, 3, 166, 2],
 [4,
  2060,
  31,
  61,
  306,
  41,
  14,
  120,
  23,
  3,
  24,
  150,
  502,
  29,
  194,
  108,
  135,
  636,
  34,
  967,
  850,
  63,
  220,
  49,
  221,
  2061,
  49,
  1618,
  63,
  220,
  49,
  221,
  2061,
  49]]

In [31]:
Y,ctokens=tokenizer(input['comment'])

In [32]:
Y[0:2]

[[43, 103, 136, 17, 122, 159, 28, 2, 1310, 4, 3, 115, 1090, 6943, 2, 6944, 1],
 [44, 1013]]

In [33]:
#padding the parents and comments
from keras.preprocessing.sequence import pad_sequences
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    padding=pad_sequences(x,padding='post',maxlen=length)
    return padding

In [34]:
X=pad(X)
Y=pad(Y)

In [35]:
X

array([[ 138,    3,  166, ...,    0,    0,    0],
       [   4, 2060,   31, ...,    0,    0,    0],
       [   9, 2383,   36, ...,    0,    0,    0],
       ...,
       [   4,   90,  400, ...,    0,    0,    0],
       [   4,   45, 1426, ...,    0,    0,    0],
       [   4,  215,   30, ...,    0,    0,    0]], dtype=int32)

In [36]:
Y

array([[  43,  103,  136, ...,    0,    0,    0],
       [  44, 1013,    0, ...,    0,    0,    0],
       [  21,   20,    5, ...,    0,    0,    0],
       ...,
       [  32, 1520,    3, ...,    0,    0,    0],
       [  26,  125,   54, ...,    0,    0,    0],
       [   3,  213,   26, ...,    0,    0,    0]], dtype=int32)